In [3]:
#%run -i read_counts.py
all_freqs = pickle.load(open('all_freqs.pkl', 'rb'))

In [4]:
from matplotlib import pyplot as plt

%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')

import pandas as pd
pd.options.display.mpl_style = 'default'

from mpltools import style
from mpltools import layout

style.use('ggplot')

## see: https://stackoverflow.com/questions/19536817/manipulate-html-module-font-size-in-ipython-notebook
class sizeme():
    """ Class to change html fontsize of object's representation"""
    def __init__(self,ob, size, height=100):
        self.ob = ob
        self.size = size
        self.height = height
    def _repr_html_(self):
        repl_tuple = (self.size, self.height, self.ob._repr_html_())
        return u'<span style="font-size:{0}%; line-height:{1}%">{2}</span>'.format(*repl_tuple)
    
## see https://stackoverflow.com/questions/14656852/how-to-use-pandas-dataframes-and-numpy-arrays-in-rpy2
## and http://ipython.org/ipython-doc/rel-0.13/config/extensions/rmagic.html
## note there's a ri2pandas() to convert back.
## but note, rpy2 2.4.0 and later automagically translates dataframes: 
## https://stackoverflow.com/questions/20630121/pandas-how-to-convert-r-dataframe-back-to-pandas
%load_ext rpy2.ipython
#import rpy2.robjects.pandas2ri as p2r
#rdf = p2r.pandas2ri(info)
#%Rpush rdf

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [5]:
x = all_freqs['Desulfovibrio_vulgaris_Hildenborough_uid57645'].copy()
sample_info = pd.read_excel('Sample_Info_COMPLETE.xlsx') ##,skiprows=[0])
sample_info = sample_info.set_index( sample_info['Sample name'] )
sample_infos = { k:sample_info.ix[all_freqs[k].columns.droplevel(1).values] for k in all_freqs.keys() }
info = sample_infos['Desulfovibrio_vulgaris_Hildenborough_uid57645'].copy()
sizeme(info.head(3),50,120)

,Sample name,Source,Barcode,Strain/condition,cultivation type,Chemostat/batch ID #,Description/condition details,Description/condition details -2,carbon source,electron donor,concentration (mM),electron acceptor,growth rate per h,T0C,organisms
Sample name,,,,,,,,,,,,,,,
Sample_US-1505885,Sample_US-1505885,Labcorp,US-1505885,PS 37 C log 2,batch,NaN,"pyruvate 80mM, sulfate 15mM 37C",sulfate respiration/growth,pyruvate 80mM,pyruvate,NaN,sulfate,NaN,37,D vulgaris Hildenborugh
Sample_US-1505888,Sample_US-1505888,Labcorp,US-1505888,LS 37C log 1,batch,NaN,"lactate 40mM, sulfate15mM 37C",sulfate respiration/growth,lactate 40mM,lactate,NaN,sulfate,NaN,37,D vulgaris Hildenborugh
Sample_US-1505895,Sample_US-1505895,Labcorp,US-1505895,LS 37C e. stat 1,batch,NaN,"lactate 40mM, sulfate15mM 37C",fermentation,lactate 10mM,lactate,10,none,0,37,D vulgaris Hildenborugh


In [6]:
%Rpush info
%Rpush x
genes = x.index.values.astype('str')
print(genes)
%Rpush genes
#%R save.image("Untitled1.RData")

['DVU0001' 'DVU0002' 'DVU0003' ..., '__too_low_aQual' '__not_aligned'
 '__alignment_not_unique']


In [7]:
%%R
#    load("Untitled1.RData")
    good_gene = which(!substr(genes,1,2)=='__' & apply(x,1,sum)>0)
    x = x[good_gene,]
    rownames(x)=genes[good_gene]
    x.orig = x = as.matrix(x); NULL

NULL


# Note I now think that we have to use all genes and use the non-centered-scaled data (ranked?) to do good classification.

## Try mProbes -- note it is very slow!
### Train classifier on growth rate

In [ ]:
%%R
require(randomForest)
source("mProbes.R")
require(doMC)
doMC:::registerDoMC(cores=8)

gr=info$growth.rate.per.h; gr=as.numeric(as.character(gr)); gr[is.na(gr)]=0.2
Y <- factor( ifelse(gr==0, 'no_growth', ifelse(gr>=0.2, 'fast_growth', 'med_growth')) )

mp.gr <- mProbes(t(x), Y, type='randomForest', ntree=1000)
head(sort(mp.gr), 20)

/home/dreiss/PYTHONUSERBASE/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: randomForest

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/home/dreiss/PYTHONUSERBASE/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: randomForest 4.6-10

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/home/dreiss/PYTHONUSERBASE/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Type rfNews() to see new features/changes/bug fixes.

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/home/dreiss/PYTHONUSERBASE/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: foreach

  res = super(Function, self).__call__(*new_args, **new_kwargs)


In [ ]:
%%R
mp.gr2 <- mProbes(t(x), Y, type='randomForest', ntree=1000)
head(sort(mp.gr2), 20)

### Train classifier on electron donor

In [ ]:
%%R
doMC:::registerDoMC(cores=8)
Y <- info$electron.donor
mp.ed <- mProbes(t(x), Y, type='randomForest', ntree=1000)
head(sort(mp.ed), 20)

### Train classifier on electron acceptor

In [ ]:
%%R
Y <- info$electron.acceptor
doMC:::registerDoMC(cores=8)
mp.ea <- mProbes(t(x), Y, type='randomForest', ntree=1000)
head(sort(mp.ea), 20)

### Train classifier on temperature

In [ ]:
%%R
Y <- info$T0C == 37
mp.temp <- mProbes(t(x), Y, type='randomForest', ntree=1000)
head(sort(mp.tmp), 20)